In [10]:
import pandas as pd
import numpy as np
import re
import datetime
import geopandas

In [11]:
df = pd.read_csv("raw_datasets/raw_shops.csv")
df = df.drop_duplicates() # to remove duplicates in case of scrapy run several times

In [12]:
df.head()

,seller_location,seller_join_date,number_of_sales,number_of_admirers,number_of_reviews,average_review_score,date_of_last_review_left,number_of_items
0,"Jaipur, India",On Etsy since 2018,0 Sales,NaN,-,-,NaN,2
1,-,On Etsy since 2017,226 Sales,NaN,(12),5,NaN,3
2,"Soorts-Hossegor, France",On Etsy since 2017,14 Sales,NaN,(2),4,NaN,0
3,-,On Etsy since 2019,0 Sales,NaN,-,-,NaN,2
4,-,On Etsy since 2019,0 Sales,NaN,-,-,NaN,9


In [13]:
df = df.drop(columns=['number_of_admirers','date_of_last_review_left'])
df.head()

,seller_location,seller_join_date,number_of_sales,number_of_reviews,average_review_score,number_of_items
0,"Jaipur, India",On Etsy since 2018,0 Sales,-,-,2
1,-,On Etsy since 2017,226 Sales,(12),5,3
2,"Soorts-Hossegor, France",On Etsy since 2017,14 Sales,(2),4,0
3,-,On Etsy since 2019,0 Sales,-,-,2
4,-,On Etsy since 2019,0 Sales,-,-,9


In [14]:
def update_nan(value):
    if value == '-':
        return np.nan
    if value == '':
        return np.nan
    return value

df = df.applymap(update_nan)
df.head()

,seller_location,seller_join_date,number_of_sales,number_of_reviews,average_review_score,number_of_items
0,"Jaipur, India",On Etsy since 2018,0 Sales,NaN,NaN,2
1,NaN,On Etsy since 2017,226 Sales,(12),5,3
2,"Soorts-Hossegor, France",On Etsy since 2017,14 Sales,(2),4,0
3,NaN,On Etsy since 2019,0 Sales,NaN,NaN,2
4,NaN,On Etsy since 2019,0 Sales,NaN,NaN,9


In [15]:
def clean_join_date(seller_join_date):
    pattern = '^On Etsy since ([0-9]+)$'
    res = re.match(pattern, seller_join_date)
    return res.group(1)
    return seller_join_date
    
def clean_number_of_sales(number_of_sales):
    pattern = '^([0-9]+) Sale'
    res = re.match(pattern, number_of_sales)
    return res.group(1)

def clean_number_of_reviews(number_of_reviews):
    pattern = '^\(([0-9]+)\)$'
    res = re.match(pattern, number_of_reviews)
    if res:
        return res.group(1)
    return np.nan

df['seller_join_date'] = df['seller_join_date'].map(clean_join_date, na_action='ignore')
df['number_of_sales'] = df['number_of_sales'].map(clean_number_of_sales, na_action='ignore')
df['number_of_reviews'] = df['number_of_reviews'].map(clean_number_of_reviews, na_action='ignore')
df.head()

,seller_location,seller_join_date,number_of_sales,number_of_reviews,average_review_score,number_of_items
0,"Jaipur, India",2018,0,NaN,NaN,2
1,NaN,2017,226,12,5,3
2,"Soorts-Hossegor, France",2017,14,2,4,0
3,NaN,2019,0,NaN,NaN,2
4,NaN,2019,0,NaN,NaN,9


In [16]:
df[df['average_review_score'] == '0'].head()

,seller_location,seller_join_date,number_of_sales,number_of_reviews,average_review_score,number_of_items
10,"Montreal, Canada",2015,1,1,0,0
15,"Oregon, United States",2013,4,2,0,0
17,NaN,2015,77,1,0,0
18,"California, United States",2015,1,1,0,0
21,NaN,2013,3,1,0,0


In [17]:
# 0 actually means that Etsy just does not show the it - so make it NaN as well
df.loc[ (df['average_review_score'] == '0'), 'average_review_score'] = np.nan


In [18]:
df.to_csv('datasets/shops.csv', index=False)

In [19]:
def parse_location(location):
    splitted = location.split(",")
    if len(splitted) == 3:
        return (splitted[0].strip(), splitted[2].strip())
    elif len(splitted) == 2:
        return (splitted[0].strip(), splitted[1].strip())
    return (np.nan, splitted[0].strip())
    
def get_town(location):
    (town, country) = parse_location(location)
    return town

def get_country(location):
    (town, country) = parse_location(location)
    return country

df['seller_town'] = df['seller_location'].map(get_town, na_action='ignore')
df['seller_country'] = df['seller_location'].map(get_country, na_action='ignore')
df.head()

,seller_location,seller_join_date,number_of_sales,number_of_reviews,average_review_score,number_of_items,seller_town,seller_country
0,"Jaipur, India",2018,0,NaN,NaN,2,Jaipur,India
1,NaN,2017,226,12,5,3,NaN,NaN
2,"Soorts-Hossegor, France",2017,14,2,4,0,Soorts-Hossegor,France
3,NaN,2019,0,NaN,NaN,2,NaN,NaN
4,NaN,2019,0,NaN,NaN,9,NaN,NaN


In [20]:
df.to_csv('datasets/shops_add.csv', index=False)

In [21]:
## prepare geocodes
wc = pd.read_csv("raw_datasets/worldcities.csv") # taken from https://www.kaggle.com/juanmah/world-cities
wc = wc[['city', 'country', 'lat', 'lng']]
wc['index'] = wc['city'] + ", " + wc['country']
wc.head()

,city,country,lat,lng,index
0,Malishevë,Kosovo,42.4822,20.7458,"Malishevë, Kosovo"
1,Prizren,Kosovo,42.2139,20.7397,"Prizren, Kosovo"
2,Zubin Potok,Kosovo,42.9144,20.6897,"Zubin Potok, Kosovo"
3,Kamenicë,Kosovo,42.5781,21.5803,"Kamenicë, Kosovo"
4,Viti,Kosovo,42.3214,21.3583,"Viti, Kosovo"


In [22]:
#get geocodes for States

# commented, as asking Nominatim takes some time
# states = geocodes[ geocodes['seller_country'] == 'United States']['seller_town'].drop_duplicates().dropna()
# states_geo = geopandas.tools.geocode(states.to_list(), provider='nominatim', user_agent="snowwlex-app")
# states_df = pd.DataFrame({
#     'state_name' : states.to_list(),
#     'lng' : states_geo['geometry'].apply(lambda p: p.x),
#     'lat' : states_geo['geometry'].apply(lambda p: p.y)
# })
# states_df.to_csv('datasets/states_geocodes.csv', index=False)

# so just load
states_df = pd.read_csv("datasets/states_geocodes.csv")
states_df['index'] = states_df['state_name'] + ", United States"
# us_geocodes_joined = geocodes.set_index('index').join(states_df.set_index('index'), how='inner')

In [23]:
#combine World Cities with States-specific data:
# as for US, it's state what is specified as 'city'
geo_locations = wc[ wc['country'] != 'United States' ][['index', 'lat', 'lng']].append(states_df[['index', 'lat', 'lng']])

In [24]:
locations = df[['seller_location', 'seller_country', 'seller_town']].drop_duplicates().dropna(subset=['seller_location'])
locations.head()

,seller_location,seller_country,seller_town
0,"Jaipur, India",India,Jaipur
2,"Soorts-Hossegor, France",France,Soorts-Hossegor
5,"Arizona, United States",United States,Arizona
8,"Milton Keynes, England",England,Milton Keynes
9,"New York, United States",United States,New York


In [25]:
locations[ locations['seller_country'] == 'United States'].head()

,seller_location,seller_country,seller_town
5,"Arizona, United States",United States,Arizona
9,"New York, United States",United States,New York
12,"Illinois, United States",United States,Illinois
13,"Oklahoma, United States",United States,Oklahoma
14,"Ohio, United States",United States,Ohio


In [26]:
#Uniting the United Kingdom
locations.loc[(locations['seller_country'] == 'England'), 'seller_country'] = 'United Kingdom'
locations.loc[(locations['seller_country'] == 'Wales'), 'seller_country'] = 'United Kingdom'
locations.loc[(locations['seller_country'] == 'Scotland'), 'seller_country'] = 'United Kingdom'
locations.loc[(locations['seller_country'] == 'Northern Ireland'), 'seller_country'] = 'United Kingdom'

locations['index'] = locations['seller_town'] + ", " + locations['seller_country']
locations.head()

,seller_location,seller_country,seller_town,index
0,"Jaipur, India",India,Jaipur,"Jaipur, India"
2,"Soorts-Hossegor, France",France,Soorts-Hossegor,"Soorts-Hossegor, France"
5,"Arizona, United States",United States,Arizona,"Arizona, United States"
8,"Milton Keynes, England",United Kingdom,Milton Keynes,"Milton Keynes, United Kingdom"
9,"New York, United States",United States,New York,"New York, United States"


In [27]:
locations_joined = locations.set_index('index').join(geo_locations.set_index('index'), how='inner')
locations_joined[ locations_joined['seller_country'] == 'United Kingdom'].sample(n=10)

,seller_location,seller_country,seller_town,lat,lng
index,,,,,
"Camberwell, United Kingdom","Camberwell, England",United Kingdom,Camberwell,51.4736,-0.0916
"Knowsley, United Kingdom","Knowsley, England",United Kingdom,Knowsley,53.4500,-2.8500
"Forfar, United Kingdom","Forfar, Scotland",United Kingdom,Forfar,56.6500,-2.8833
"Ipswich, United Kingdom","Ipswich, England",United Kingdom,Ipswich,52.0703,1.1700
"Enniskillen, United Kingdom","Enniskillen, Northern Ireland",United Kingdom,Enniskillen,54.3466,-7.6411
"Shrewsbury, United Kingdom","Shrewsbury, England",United Kingdom,Shrewsbury,52.7167,-2.7333
"Milton Keynes, United Kingdom","Milton Keynes, England",United Kingdom,Milton Keynes,52.0333,-0.7000
"Haddington, United Kingdom","Haddington, Scotland",United Kingdom,Haddington,55.9333,-2.7667
"Torquay, United Kingdom","Torquay, England",United Kingdom,Torquay,50.4500,-3.5000


In [28]:
locations_joined[ locations_joined['seller_country'] == 'United States'].sample(n=10)

,seller_location,seller_country,seller_town,lat,lng
index,,,,,
"Delaware, United States","Delaware, United States",United States,Delaware,38.692045,-75.401331
"Maryland, United States","Maryland, United States",United States,Maryland,39.516223,-76.938207
"Oregon, United States","Oregon, United States",United States,Oregon,43.979280,-120.737257
"Georgia, United States","Georgia, United States",United States,Georgia,32.329381,-83.113737
"Kentucky, United States","Kentucky, United States",United States,Kentucky,37.572603,-85.155141
"Massachusetts, United States","Massachusetts, United States",United States,Massachusetts,42.378877,-72.032366
"Pennsylvania, United States","Pennsylvania, United States",United States,Pennsylvania,40.969989,-77.727883
"Minnesota, United States","Minnesota, United States",United States,Minnesota,45.989659,-94.611329
"Alabama, United States","Alabama, United States",United States,Alabama,33.258882,-86.829534


In [29]:
locations_joined[ locations_joined['seller_country'] == 'United States']

,seller_location,seller_country,seller_town,lat,lng
index,,,,,
"Alabama, United States","Alabama, United States",United States,Alabama,33.258882,-86.829534
"Alaska, United States","Alaska, United States",United States,Alaska,64.445961,-149.680909
"Arizona, United States","Arizona, United States",United States,Arizona,34.395342,-111.763275
"Arkansas, United States","Arkansas, United States",United States,Arkansas,35.204888,-92.447911
"California, United States","California, United States",United States,California,36.701463,-118.755997
"Colorado, United States","Colorado, United States",United States,Colorado,38.725178,-105.607717
"Connecticut, United States","Connecticut, United States",United States,Connecticut,41.650020,-72.734216
"Delaware, United States","Delaware, United States",United States,Delaware,38.692045,-75.401331
"Florida, United States","Florida, United States",United States,Florida,27.756767,-81.463983


In [30]:
all_locations = locations_joined.reset_index()[['seller_location', 'lng', 'lat']]
all_locations.head()

,seller_location,lng,lat
0,"Aalborg, Denmark",9.9166,57.0337
1,"Aarau, Switzerland",8.0524,47.3896
2,"Aarau, Switzerland",8.0340,47.3900
3,"Abakan, Russia",91.4450,53.7037
4,"Abaza, Russia",90.0954,52.6690


In [31]:
all_locations.to_csv('datasets/all_locations.csv', index=False)